# AI6122 Text Management Review Summarizer
Task List:
- [ ] Define and justify what a summarizer is. E.g.
  + a list of keywords
  + a list of key phrases
  + a list of noun-adjective pairs 
  + a list of nounPhrase - adjectivePhrase pairs 
  + a list of representative sentences 
- [ ] Technical challenges to achieve ideal summarization and your solution.
- [ ] Justify approach is best option for each component in your solution.
- [ ] Justify limitations to your approach.
- [ ] Evaluate solution with possible alternative solutions (baselines).
- [ ] Randomly choose 3 products to create product review summary.

Possible algorithms to try:
- [x] (Extractive) Word frequency sentence extraction: score sentences by word freq
- (Extractive) Text rank algorithm: uses cosine similarity
- (Extractive/ Abstractive) Machine learning 



## Colab Configuration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os 
os.chdir('/content/drive/My Drive/Colab_Notebooks/text_management/SourceCode/02 Review Summarizer/')
!pwd

/content/drive/My Drive/Colab_Notebooks/text_management/SourceCode/02 Review Summarizer


## Import Modules & Configurations

In [0]:
from collections import OrderedDict
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
import multiprocessing as mp
import datetime
from helpers.duallogger import loggersetup
from helpers.filehelper import is_not_empty_file_exists, write_to_file, load_from_file
import logging

from nltk.corpus import stopwords
import nltk
import re
import heapq

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
stopwords_nltk = stopwords.words('english')
stopwords_spacy = list(STOP_WORDS)
stopwords_spacy.append('\n')
stopwords = stopwords_nltk + list(set(stopwords_spacy) - set(stopwords_nltk))

print("sw nltk: ", len(stopwords_nltk))
print("sw spacy: ", len(stopwords_spacy))
print("combined: ", len(stopwords))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
sw nltk:  179
sw spacy:  327
combined:  383


In [11]:
cores = mp.cpu_count()
print("Cores:", cores)
gpu = spacy.prefer_gpu()
print("GPU:", gpu)

log_dir = './logs/'
log = loggersetup(log_dir, stdout_level=logging.DEBUG, file_level=logging.DEBUG)

# log.debug('Debug message, should only appear in the file.')
# log.info('Info message, should appear in file and stdout.')
# log.warning('Warning message, should appear in file and stdout.')
# log.error('Error message, should appear in file and stdout.')

Cores: 2
GPU: False


In [0]:
parameters = OrderedDict()
parameters['json_file'] = 'CellPhoneReview.json'
parameters['reload_prod_reviews'] = True
parameters['prod_reviews_path'] = './data/prod_reviews.data'
parameters['clean_reviews'] = True
parameters['reload_clean_reviews'] = True
parameters['cleaned_reviews_path'] = './data/prod_reviews_cleaned.data'
parameters['reload_collection_frequencies'] = True
parameters['collection_frequencies_path'] = './data/word_doc_frequencies.data'

## Data Preprocessing

* convert_lower_case(data)
* lemma(data)
* remove_punctuation(data)
* remove_stop_words(data)

In [8]:
if not parameters['reload_prod_reviews'] or not is_not_empty_file_exists(parameters['prod_reviews_path']):
    data = pd.read_json(parameters['json_file'], lines = True)
    prod_reviews = data.groupby(['asin'])['reviewText'].apply(' '.join).reset_index()
    log.info("Writing prod_reviews to %s" % parameters['prod_reviews_path'])
    write_to_file(parameters['prod_reviews_path'], prod_reviews)
else:
    log.info("Reloading prod_reviews from %s" % parameters['prod_reviews_path'])
    prod_reviews = load_from_file(parameters['prod_reviews_path'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
prod_reviews.head(3) 

[INFO] Writing prod_reviews to ./data/prod_reviews.data


asin  \
0  120401325X   
1  3998899561   
2  6073894996   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [18]:
# Create the nlp object
nlp = spacy.load("en_core_web_sm")

# def set_custom_boundaries(doc):
#     for i, token in enumerate(doc):
#         if token.text in ("but"):
#             doc[i].is_sent_start = False
#         if token.text in ("’s", "'s"):
#             doc[i].is_sent_start = False
#         elif token.text in ("“", "‘") and i < len(doc) - 1:
#             # opening quote
#             doc[i+1].is_sent_start = False
#         elif token.text in ("”", "’"):
#             # closing quote
#             doc[i].is_sent_start = False
#     return doc

# nlp.add_pipe(set_custom_boundaries, before="parser")

print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [10]:
sentnlp = spacy.load("en_core_web_sm", disable=['parser', 'ner', 'tagger'])
sentnlp.add_pipe(nlp.create_pipe('sentencizer'))

# Define function to cleanup text by removing personal pronouns, stopwords, and punctuations
def cleanup_text(text, stopwords, punc):
    texts = []
    global sentnlp
    doc = sentnlp(text) # only do tokenization and pos tagging
    for sentence in doc.sents:
        sen_text = nlp(sentence.text, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in sen_text if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punc]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return str(texts)

def tokenize_sentences(text): 
    texts = []
    global sentnlp
    doc = sentnlp(text) # only do tokenization and pos tagging
    for idx, sentence in enumerate(doc.sents):
        texts.append(sentence.text)
    return str(texts)

if parameters['clean_reviews']:
    if not parameters['reload_clean_reviews'] or not is_not_empty_file_exists(parameters['cleaned_reviews_path']):
        punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'
        log.debug("reviews to be stripped away with stopwords and punctation")
        prod_reviews['processedSentences'] = prod_reviews['reviewText'].apply(lambda x: cleanup_text(x, stopwords, punctuations))
        prod_reviews['reviewSentences'] = prod_reviews['reviewText'].apply(lambda x: tokenize_sentences(x))
        log.info("Writing cleaned reviews to %s" % parameters['cleaned_reviews_path'])
        write_to_file(parameters['cleaned_reviews_path'], prod_reviews)
    else:
        log.info("Reloading cleaned reviews from %s" % parameters['cleaned_reviews_path'])
        prod_reviews = load_from_file(parameters['cleaned_reviews_path'])
        
else:
    log.info("unprocessed reviews will be used")
    prod_reviews['reviewSentences'] = prod_reviews['reviewText'].apply(lambda x: tokenize_sentences(x))
    prod_reviews['processedSentences'] = prod_reviews['reviewSentences']

print(prod_reviews['processedSentences'][7])
print(prod_reviews['reviewSentences'][7])

[DEBUG] reviews to be stripped away with stopwords and punctation


KeyboardInterrupt: ignored

In [0]:
# load cleaned data from file
prod_reviews = load_from_file(parameters['cleaned_reviews_path'])

In [8]:
print(prod_reviews['processedSentences'][7])
print(prod_reviews['reviewSentences'][7])

['great product order future .', 'money spend', 'need great car charger need', 'light port little rough charge phone fine .', 'wish little reach use phone passenger seat car comfortably big problem .', 'reach pretty far coil section stress charge port phone people realize eventually break .', 'purchase charger sister girlfriend .', 'samsung galaxy phone need car charger oem fanboy swoop couple remain slightly skeptical actually mail price .', 'happy report definitely receive 2 oem samsung car charger mail seal bag barcode kind normally receive new refurb phone .', 'wrong price certainly argue benefit oem vs. generic output reliability list .', 'buy samsung charge .', 'work perfectly build solidly 20 verizon charger .', 'highly recommend car charger thank great price', 'need pleased .', 'buy cheap lame .', 'charger work great .', 'issue .', 'wish retractable cord like charger use love .']
['This is another great product and i will always order from here in the future.', 'money spent wel

## Review Summarizer

### Collection Frequency

In [12]:
import numpy as np
def collection_frequencies(prod_reviews, use_cleaned) -> dict:
    collection_frequencies = {}
    for index, row in prod_reviews.iterrows():
        seen_tokens = set()
        if (use_cleaned):
            reviews = ' '.join(str(x) for x in eval(row['processedSentences'])) 
        else:
            reviews = ' '.join(str(x) for x in eval(row['reviewSentences'])) 
        for token in nlp.make_doc(reviews):   #tokenize
            if token.text in seen_tokens:
                continue
            if token.text not in stopwords:
                if token.text not in collection_frequencies.keys():
                    collection_frequencies[token.text] = 1
                else:
                    collection_frequencies[token.text] += 1
            seen_tokens.add(token.text)
    return collection_frequencies

if not parameters['reload_collection_frequencies'] or not is_not_empty_file_exists(parameters['collection_frequencies_path']):
    word_doc_frequencies = collection_frequencies(prod_reviews, parameters['clean_reviews'])
    log.info("Writing collection frequencies to %s" % parameters['collection_frequencies_path'])
    write_to_file(parameters['collection_frequencies_path'], word_doc_frequencies)
else:
    log.info("Reloading collection frequencies from %s" % parameters['collection_frequencies_path'])
    word_doc_frequencies = load_from_file(parameters['collection_frequencies_path'])

print(word_doc_frequencies)
print(type(word_doc_frequencies))

Output hidden; open in https://colab.research.google.com to view.

In [13]:
len(word_doc_frequencies)

139844

### Tokenize Words & Calculate Word Frequency 

Only calculate word frequencies of non-stopwords



In [0]:
def _word_frequency(review) -> dict: 
  global stopwords 
  word_frequencies = {}  
  for token in nlp.make_doc(review):   #tokenize
      if token.text not in stopwords:
          if token.text not in word_frequencies.keys():
              word_frequencies[token.text] = 1
          else:
              word_frequencies[token.text] += 1
  return word_frequencies

# wf = _word_frequency(prod_reviews['reviewText'][8])
# print(wf)

### Tokenize Sentences & Score Sentences

In [0]:
def _tokenize_sentences(review) -> dict:
  global sentnlp
  doc = sentnlp(review)
  sentence_dict = {}
  for _, sentence in enumerate(doc.sents): 
    tokenized_sent = re.sub(r'[^\w\s]','', str(sentence))
    if tokenized_sent is not None:
      sentence_dict[tokenized_sent.strip()] = len(nlp.make_doc(sentence.text))
  return sentence_dict

# sent_dict = _tokenize_sentences(prod_reviews['reviewText'][8])
# print([(key, value) for key, value in sent_dict.items()])

# def _score_sentences(sentences:dict, word_frequencies:dict) -> dict:
#     """Score sentences based on word frequencies"""
#     sentence_scores = {}

#     for sent, word_count_in_sent in sentences.items():
#         for word_freq in word_frequencies:
#             if word_freq in sent.lower():
#                 if sent in sentence_scores: # use first 10 char as key
#                     sentence_scores[sent] += word_frequencies[word_freq]
#                 else:
#                     sentence_scores[sent] = word_frequencies[word_freq]
        
#         if sent in sentence_scores: # divide sentence score by word count to reduce advantages of long sentences
#           sentence_scores[sent] = sentence_scores[sent] / word_count_in_sent

#     return sentence_scores

# sent_scores = _score_sentences(sent_dict, wf)
# print([(key, value) for key, value in sent_scores.items()])

### Sentence-level Summarizer


In [16]:
  index = 10
  processedSentences = eval(prod_reviews['processedSentences'][index])
  reviewSentences = eval(prod_reviews['reviewSentences'][index])

  print(len(processedSentences))
  print(len(reviewSentences))

22
22


In [22]:
from nltk.tokenize import sent_tokenize

from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
  return dot(a, b)/(norm(a)*norm(b))

def generate_summary(prod_reviews, index, num_of_docs, word_doc_frequencies, num_of_sentences):
    processedSentences = eval(prod_reviews['processedSentences'][index])
    reviewSentences = eval(prod_reviews['reviewSentences'][index])

    ## generate word frequencies in cleaned
    cleaned_reviews = ' '.join(str(x) for x in processedSentences) 
    word_frequencies = _word_frequency(cleaned_reviews)

    ## generate review vector of len of word frequencies
    vocab = len(word_frequencies)
    # log.debug("vocab: %i" % vocab)
    review_tf_vec = np.zeros(len(word_frequencies))
    for idx, word in enumerate(word_frequencies):
        tfidf_value = (1 + np.log(word_frequencies[word])) * np.log(num_of_docs/word_doc_frequencies[word]) 
        review_tf_vec[idx] = tfidf_value

    # print(review_tf_vec)
    log.debug(len(review_tf_vec))

    ## generate sentences
    # log.debug("num of sentences: %i" % len(processedSentences))
    # log.debug("sentences: %s" % processedSentences)

    ## generate sentence vectors of len of word frequencies
    sent_tf_vecs = []
    for sentence in processedSentences:
        sent_tf_vec = np.zeros(len(word_frequencies))
        sent_word_frequencies = _word_frequency(sentence)
        for idx, word in enumerate(sent_word_frequencies):
            if word in word_frequencies.keys():
                idx = list(word_frequencies.keys()).index(word)
                tfidf_value = (1 + np.log(sent_word_frequencies[word])) * np.log(num_of_docs/word_doc_frequencies[word]) 
                sent_tf_vec[idx] = tfidf_value
        sent_tf_vecs.append(sent_tf_vec)

    ## compare cossim between review vector and sentence vectors
    cosine_similarity = []
    for sent_tf_vec in sent_tf_vecs:
        cosine_similarity.append(cos_sim(review_tf_vec, sent_tf_vec))

    highest_cos_idx = np.argsort(cosine_similarity)[-num_of_sentences:]
    # log.debug("highest_cos_idx: %s" % str(highest_cos_idx))

    ## original sentences
    # log.debug("original_sentences: %s" % reviewSentences)
    # log.debug("num of ori sentences: %s" % len(reviewSentences))
    summary_sentences = []
    for idx in highest_cos_idx:
        # log.debug("idx: %i ori: %s cleaned: %s" % (idx, reviewSentences[idx], processedSentences[idx]))
        # log.debug("cos_vec: %s" % str(sent_tf_vecs[idx]))
        summary_sentences.append(reviewSentences[idx])

    return " ".join(summary_sentences), summary_sentences

index=10
summary, summary_sent = generate_summary(prod_reviews, index=index, num_of_docs=len(prod_reviews), word_doc_frequencies=word_doc_frequencies, num_of_sentences=3)
log.info("===========================================================================================================================")
log.info("SUMMARIZED: %s" % summary)
log.info("ORIGINAL: %s" % prod_reviews['reviewText'][index])

[DEBUG] 70
[INFO] ===========================================================================================================================
[INFO] SUMMARIZED: case protects well and the color is awesome nice The color is amazing all of my friends are jealous and ask me where did I get it. This PowerBear case is a lifesaver!! Now my phone is charged anywhere anytime thanks to powerbear .
[INFO] ORIGINAL: charges me up all the way. doubles my battery liek promised. case protects well and the color is awesome nice The color is amazing all of my friends are jealous and ask me where did I get it. They are even more jealous when I tell them how much I paid for it! Great case, powers me up and VERY stylish. Matches my 5C PERFECTLY :) much slimmer than I thought and it ads 125% battery. I am charged for the full day with this case. Great quality product , I love this case it is the perfect sidekick to an always dyeing phone . Now my phone is charged anywhere anytime thanks to powerbear . Rea

In [42]:
###### Kmeans algorithm ################


from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

def kmeans_summary(prod_reviews, index, num_of_docs, word_doc_frequencies, num_of_sentences):

    processedSentences = eval(prod_reviews['processedSentences'][index])
    reviewSentences = eval(prod_reviews['reviewSentences'][index])

    ## generate word frequencies in cleaned
    cleaned_reviews = ' '.join(str(x) for x in processedSentences) 
    word_frequencies = _word_frequency(cleaned_reviews)

    sent_tf_vecs = []
    for sentence in processedSentences:
        sent_tf_vec = np.zeros(len(word_frequencies))
        sent_word_frequencies = _word_frequency(sentence)
        for idx, word in enumerate(sent_word_frequencies):
            if word in word_frequencies.keys():
                idx = list(word_frequencies.keys()).index(word)
                tfidf_value = (1 + np.log(sent_word_frequencies[word])) * np.log(num_of_docs/word_doc_frequencies[word]) 
                sent_tf_vec[idx] = tfidf_value
        sent_tf_vecs.append(sent_tf_vec)

    est = KMeans(n_clusters=num_of_sentences, random_state=0).fit(np.array(sent_tf_vecs))

    closest_centers_idx = []
    indexes = np.arange(len(sent_tf_vecs))
    cluster_ids = est.labels_
    centroids = est.cluster_centers_

    # calculate each sentence's dist from its corresponding centroid 
    dist_from_centroids = []
    for idx in range(len(sent_tf_vecs)):
        dist = euclidean_distances(sent_tf_vecs[idx].reshape(1,-1), centroids[cluster_ids[[idx]], :])
        dist_from_centroids.append(dist[0][0])

    dist_arr = np.concatenate([np.array([dist_from_centroids]), cluster_ids.reshape(1,-1), indexes.reshape(1,-1)], axis = 0 )
    dist_arr = dist_arr.T

    # find sentences with shortest dist to centroids
    for cluster_id in range(len(centroids)):
        dist_arr_id = dist_arr[dist_arr[:, 1] == cluster_id]
        row_idx = dist_arr_id[:, 0].argmin()
        sent_idx = dist_arr_id[row_idx][-1]
        closest_centers_idx.append(sent_idx)
    closest_centers_idx = sorted(closest_centers_idx)
    print('SELECTED SENT index:', closest_centers_idx)

    print('\n')
    print('[ORIGINAL]:')
    for sent in reviewSentences:
        print(sent)

    #Comparing against vector space model
    _, vec_sent = generate_summary(prod_reviews, index=index, num_of_docs=len(prod_reviews), word_doc_frequencies=word_doc_frequencies, num_of_sentences=num_of_sentences)
    print('\n')
    print('[SUMMARY Vector Space]:')
    for i in vec_sent:
        print(i)

    summary_sent = []
    print('\n')
    print('[SUMMARY Kmeans]:')
    for idx in closest_centers_idx:
        print(reviewSentences[int(idx)])
        summary_sent.append(reviewSentences[int(idx)])
    
    return " ".join(summary_sent)

index = 150
num_of_sentences = 4
summary = kmeans_summary(prod_reviews, index=index, num_of_docs=len(prod_reviews), word_doc_frequencies=word_doc_frequencies, num_of_sentences=num_of_sentences)

SELECTED SENT index: [17.0, 18.0, 19.0, 23.0]


[ORIGINAL]:
I agree with other reviews about the design features and charging options but those things come secondary if static over takes them.
The static is real even the phone is only inches away from the unit.
Unlike my earlier Bluetooth devices this one has really short range...like 3 feet.
I can not live with static so this one is going back.
Sorry Blueant.
This is the best headset I have every had.
 The comfort is great and so is the clarity.
 The fact that it comes with so much makes it unlike any other.
 You normally have to purchase all the accessories but they all came with it.
 In my opinion, it's your best buy.
This headset is exactly as described by the manufacturer.
The sound is great.
The fit is very comfortable; I have had it on for 2-3 hours without even realizing it's on.
You can't go wrong with this.
Looks very cool.
 Very convenient charging via USB port.
 Not so good sound quality.
 Audio has a lot of static tics and

In [0]:
# def generate_summary(review, num_of_sentences):
#     word_frequencies = _word_frequency(review)

#     sentences = _tokenize_sentences(review)

#     sentence_scores = _score_sentences(sentences, word_frequencies)
    
#     summary_sentences = heapq.nlargest(num_of_sentences, sentence_scores, key=sentence_scores.get)
    
#     return ''.join(summary_sentences)

# idx = 3
# num_of_sentences = 7
# if parameters['clean_reviews']:
#     review = prod_reviews['processedText'][idx]
    
# else:
#     review = prod_reviews['reviewText'][idx]

# summary = generate_summary(review, num_of_sentences)
# log.info("ORIGINAL REVIEW: %s", review)
# log.info("SUMMARIZED REVIEW: %s", summary)

[INFO] ORIGINAL REVIEW: love case pretty . love way case feel touch rubber . happy idea design sweet idea wear paint case sealant . cool 3d effect cost paint rub . pretty . worried order picture description page change desireable green orange cover . alas come day earlier expect beautiful pink product expect rubberized cover feel little greasy like armor all'd design gorgeous little 3d look pretty phone case love . love change .thanks case white silver . pretty case- fit phone perfectly . long arrive . defintiely worth price ... order . case 15 especially cell phone store . believe mark thing great case ... fit phone perfectly use time time . recommend received item quickly . design vivid expect . cover soft rubberize durable . receive compliment . excellent buy recommend want good look phone . cover old phone look feel new . like order cover little money snazzy phone .
[INFO] SUMMARIZED REVIEW: look pretty phone case love love case pretty pretty case fit phone perfectly cover old phon